---
### 4.2.1  整数线性规划模型
> 下面均为0-1规划模型
#### 1. 背包问题
$$
\begin{align*}
&{\rm max}\ z=\sum\limits_{i=1}^nv_ix_i,\\
&{\rm s.t.}
	\begin{cases}
	\sum\limits_{i=1}^nw_ix_i\leq W,\\
	x_i=0\,\text{或}\, 1, \quad i=1, 2,\cdots, n.
    \end{cases}
\end{align*}
$$

#### 2. 指派问题
$$
\begin{align*}
&{\rm min}\ z=\sum\limits_{i=1}^n \sum\limits_{j=1}^nc_{ij}x_{ij},\\
&{\rm s.t.} 
	\begin{cases}
	\sum\limits_{j=1}^nx_{i_0j}=1, \quad i_0=1,2,\cdots,n,\\
	\sum\limits_{i=1}^nx_{ij_0}=1, \quad j_0=1,2,\cdots,n,\\
	x_{ij}=0\,\text{或}\,1, \quad i,j=1,2,\cdots, n.
	\end{cases}
\end{align*}
$$

#### 3. 旅行商问题
$$
\begin{align*}
&{\rm min}\ z=\sum\limits_{i=1}^n \sum\limits_{j=1}^nc_{ij}x_{ij},\\
&{\rm s.t.}
	\begin{cases}
	\sum\limits_{j=0}^nx_{i_0j}=1,\quad i_0=1,2,\cdots,n,\\
    \sum\limits_{i=0}^nx_{ij_0}=1,\quad j_0=1,2,\cdots,n,\\
	u_i-u_j+nx_{ij}\leq n-1, \quad i=1,2,\cdots,n,\ j=2,\cdots,n\\
    u_1=0, \ 1\leq u_i\leq n-1, \quad i=2,3,\cdots,n, \\
    x_{ij}=0\,\text{或}\,1, \quad i,j=1,2,\cdots, n.
	\end{cases}
\end{align*}
$$

### 4.2.2 整数线性规划模型的求解
#### 1. 工时安排问题

In [1]:
import cvxpy as cp

demand = [35, 40, 50, 45, 55, 30]
x = cp.Variable(6, integer=True)
obj = cp.Minimize(cp.sum(x))
cons = [
    x[0] + x[5] >= demand[0],
    x[1] + x[0] >= demand[1],
    x[2] + x[1] >= demand[2],
    x[3] + x[2] >= demand[3],
    x[4] + x[3] >= demand[4],
    x[5] + x[4] >= demand[5],
    x >= 0
]
prob = cp.Problem(obj, cons)
prob.solve(solver='GLPK_MI')
print(f'最优解为：{x.value}'), print(f'最优值为：{prob.value}')

最优解为：[35.  5. 45. 25. 30.  0.]
最优值为：140.0


(None, None)

#### 2. 装修任务分配问题

In [2]:
import cvxpy as cp
import numpy as np

cost = np.loadtxt('../../04第4章  线性规划和整数规划模型/data4_10.txt')
x = cp.Variable((4,5), integer=True)
obj = cp.Minimize(cp.sum(cp.multiply(cost, x)))
cons = [
    cp.sum(x, axis=0) == 1,
    cp.sum(x, axis=1) <= 2,
    0 <= x,
    x <= 1
]
prob = cp.Problem(obj, cons)
prob.solve(solver='GLPK_MI')
print(f'最优解为：{x.value}'), print(f'最优值为：{prob.value}')

最优解为：[[0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
最优值为：63.8


(None, None)

#### 3. 网点覆盖问题

In [3]:
import cvxpy as cp
import numpy as np
from sklearn.metrics import euclidean_distances # 两个矩阵的成对平方欧氏距离

pos = np.loadtxt('../../04第4章  线性规划和整数规划模型/data4_11.txt')
pos = pos.T
d = euclidean_distances(pos, pos)
d

array([[ 0.        ,  4.75629919,  2.74059248,  2.42243697,  5.73532972,
        11.98693418,  0.56479575,  7.17310289,  6.77562974, 11.6105781 ],
       [ 4.75629919,  0.        ,  7.03959836,  6.56771655,  3.14550105,
         7.35275694,  4.25236497,  4.58762493,  6.84323888,  8.52356222],
       [ 2.74059248,  7.03959836,  0.        ,  0.50409771,  8.45513224,
        13.84437886,  3.28587179,  8.07617252,  6.27769905, 12.30855942],
       [ 2.42243697,  6.56771655,  0.50409771,  0.        ,  8.08048245,
        13.3413371 ,  2.94304882,  7.58945459,  5.9106389 , 11.84379706],
       [ 5.73532972,  3.14550105,  8.45513224,  8.08048245,  0.        ,
         8.75631644,  5.17661654,  7.69877494,  9.85159815, 11.35906394],
       [11.98693418,  7.35275694, 13.84437886, 13.3413371 ,  8.75631644,
         0.        , 11.53122036,  6.67302968, 10.4321443 ,  6.06331822],
       [ 0.56479575,  4.25236497,  3.28587179,  2.94304882,  5.17661654,
        11.53122036,  0.        ,  6.94650153

In [4]:
x = cp.Variable(10, integer=True)
y = cp.Variable((10,10), integer=True)
obj = cp.Minimize(cp.sum(x))
cons = [
    cp.sum(y, axis=0) >= 1,
    cp.sum(y, axis=1) <= 5,
    0 <= x, x <= 1,
    0 <= y, y <= 1
]
for i in range(10):
    cons.append(x[i] == y[i,i])
    for j in range(10):
        cons.append(d[i,j]*y[i,j] <= 10*x[i])
        cons.append(x[i] >= y[i,j])
prob = cp.Problem(obj, cons)
prob.solve(solver='GLPK_MI')
print(f'最优解为：{x.value}\n{y.value}'), print(f'最优值为：{prob.value}')

最优解为：[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
最优值为：2.0


(None, None)